# Amazon Review Clustering approach 2



In [4]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error
import tarfile
import json
import seaborn as sns  
import matplotlib.pyplot as plt 
import pandas as pd
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from pathlib import Path


In [5]:
path = Path('/content/drive/MyDrive/Colab Notebooks/data_1pct.json')

print(path.is_file())

True


In [6]:
df_data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data_1pct.json', lines=True, orient='records')



```
# This is formatted as code
```

# Preprocessing

In [7]:
df_new = df_data.loc[:, ['reviewText', 'summary']]
df_new['reviewText'] = df_new['reviewText'].replace(np.nan, '')

In [8]:
nltk.download('maxent_ne_chunker')
nltk.download('words')


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [11]:
# Download required NLTK resources
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load stop words
stop_words = set(stopwords.words('english'))

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:

# Define function to preprocess text
def preprocess_text(text):
    # Tokenize text into sentences
    sentences = sent_tokenize(text)
    # Tokenize sentences into words and tag parts of speech
    tagged_words = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        tagged = nltk.pos_tag(words)
        tagged_words.extend(tagged)
    # Extract noun phrases
    chunks = nltk.ne_chunk(tagged_words, binary=False)
    phrases = []
    for chunk in chunks:
        if hasattr(chunk, 'label') and chunk.label() == 'NP':
            phrases.append(' '.join([c[0] for c in chunk]))
    # Remove stop words and lemmatize words
    words = [lemmatizer.lemmatize(word.lower()) for word in phrases if word.lower() not in stop_words and len(word) > 2]
    # Join words back into text
    text = ' '.join(words)
    return text


In [ ]:
# Apply preprocessing to 'reviewText' and 'summary' columns
df_new['reviewText'] = df_new['reviewText'].apply(preprocess_text)
df_new['summary'] = df_new['summary'].apply(preprocess_text)



```
# This is formatted as code
```

# Applying LDA and calculating Coherence

In [ ]:
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

In [ ]:
# Convert the preprocessed text to a list of words
docs = df_new['reviewText'].apply(lambda x: x.split())

# Create a dictionary of the words in the corpus
dictionary = Dictionary(docs)

# Filter out words that appear in less than 10 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=10, no_above=0.5)

# Convert the corpus to a bag of words format
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Choose the number of topics for the LDA model
num_topics = 10

# Fit the LDA model to the corpus
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10)


Model Evaluation

In [ ]:
# Evaluate the model's performance using coherence score
coherence_model_lda = CoherenceModel(model=lda, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

# Inspect the topics
topics = lda.show_topics(num_topics=num_topics, formatted=False)
for i, topic in enumerate(topics):
    print('Topic {}: {}'.format(i, [word[0] for word in topic[1]]))